# Model Registry

In [6]:
from snowflake.snowpark import Session, Column, functions
from snowflake.ml.registry import model_registry

In [1]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session

In [4]:
session = get_session()
# session.close()
session

In [5]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWFRACT"
REGISTRY_SCHEMA_NAME = "SNOWFRACT"

### Create a registry if not created already

In [7]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


True

### Connect to registry

In [8]:
registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [9]:
registry.get_history().show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"EVENT_TIMESTAMP"                 |"EVENT_ID"                        |"MODEL_ID"                        |"ROLE"          |"OPERATION"  |"ATTRIBUTE_NAME"  |"VALUE[ATTRIBUTE_NAME]"                             |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2023-08-03 11:37:47.031000+00:00  |2b1437f231f211eeab49c6770d61e4f3  |29da402031f211eeab49c6770d61e4f3  |"ACCOUNTADMIN"  |SET          |REGISTRATION      |{                                                   |
|                                  |                                  |                                  |                |             |                  |  "C

In [10]:
MODEL_NAME="ChurnPrediction"
MODEL_VERSION="ee9215ef16a2c0fe34d17f557600a36c"

/# A name and model tags can be added to the model at registration time.

<code>model_id = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=pipeline,
    tags={"stage": "testing", "classifier_type": "xgboost"},
    sample_input_data=test_df.show(),
    options={"embed_local_ml_library": True}
)</code>

In [11]:
model_id = "200e68963ff311ee89fa628aca6d24dc"

In [14]:
model_list = registry.list_models()

In [15]:
model_list.filter(model_list["ID"] == model_id).select("NAME","TAGS","METRICS").show()

----------------------------------------------------------------------------------------------
|"NAME"           |"TAGS"                           |"METRICS"                               |
----------------------------------------------------------------------------------------------
|ChurnPrediction  |{                                |{                                       |
|                 |  "classifier_type": "xgboost",  |  "build_time_metrics": {               |
|                 |  "stage": "testing"             |    "accuracy": 0.8448098663926003,     |
|                 |}                                |    "exit": {                           |
|                 |                                 |      "f1-score": 0.5251572327044026,   |
|                 |                                 |      "precision": 0.6398467432950191,  |
|                 |                                 |      "recall": 0.44533333333333336,    |
|                 |                               

In [19]:
pipeline_restored = registry.load_model(MODEL_NAME,MODEL_VERSION)

In [20]:
type(pipeline_restored)

snowflake.ml.modeling.pipeline.pipeline.Pipeline

In [23]:
ppln_loaded = model_registry.ModelReference(registry=registry, model_name=MODEL_NAME,model_version=MODEL_VERSION)

In [25]:
ppln_loaded.deploy(
    deployment_name="churn_model_predict",
    target_method="predict",
    permanent=True,
    options={"relax_version":True}
)

Generated UDF file is persisted at: /tmp/tmp0dorw3fc.py


"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."churn_model_predict" is deployed to warehouse.


### Following queries got executed in snowflake on deployment

<code>1) CREATE STAGE IF NOT EXISTS "MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"._SYSTEM_REGISRTRY_DEPLOYMENTS_STAGE
</code>

<code>2) SELECT  *  FROM (SELECT * FROM "MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."_SYSTEM_REGISTRY_MODELS_VIEW") WHERE (("NAME" = 'ChurnPrediction') AND ("VERSION" = 'ee9215ef16a2c0fe34d17f557600a36c'))
</code>

<code>3) LIST @"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT".SNOWML_MODEL_200E68963FF311EE89FA628ACA6D24DC
</code>

<code>4) SELECT  *  FROM (SELECT * FROM "MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."_SYSTEM_REGISTRY_MODELS_VIEW") WHERE (("NAME" = 'ChurnPrediction') AND ("VERSION" = 'ee9215ef16a2c0fe34d17f557600a36c'))
</code>

<code>5) GET '@"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT".snowml_model_200e68963ff311ee89fa628aca6d24dc/tmpqfs1lw9d.zip' 'file:///tmp'  parallel = 10
</code>

<code>6) ls '@"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"._SYSTEM_REGISRTRY_DEPLOYMENTS_STAGE/churn_model_predict'
</code>

<code>7) SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01ae73bc-0503-b103-0072-f3030ba6b46a')))
</code>

<code>8) SELECT "PACKAGE_NAME", array_agg("VERSION") AS "ARRAY_AGG(VERSION)" FROM ( SELECT  *  FROM information_schema.packages WHERE (("LANGUAGE" = 'python') AND "PACKAGE_NAME" IN ('anyio', 'cloudpickle', 'numpy', 'packaging', 'pandas', 'pyyaml', 'scikit-learn', 'snowflake-snowpark-python', 'typing-extensions', 'xgboost'))) GROUP BY "PACKAGE_NAME"
</code>

<code>9) PUT 'file:///tmp/tmp0dorw3fc.py' '@"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"._SYSTEM_REGISRTRY_DEPLOYMENTS_STAGE/churn_model_predict/MODEL_REGISTRY_SNOWFRACTSNOWFRACTchurn_model_predict_8980348272667569519' PARALLEL = 4 AUTO_COMPRESS = FALSE SOURCE_COMPRESSION = AUTO_DETECT OVERWRITE = TRUE
</code>

<code>10) CREATE
  FUNCTION  "MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."churn_model_predict"(arg1 OBJECT)
RETURNS OBJECT
LANGUAGE PYTHON 
RUNTIME_VERSION=3.8
IMPORTS=('@"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT".snowml_model_200e68963ff311ee89fa628aca6d24dc/tmpqfs1lw9d.zip','@"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"._SYSTEM_REGISRTRY_DEPLOYMENTS_STAGE/churn_model_predict/MODEL_REGISTRY_SNOWFRACTSNOWFRACTchurn_model_predict_8980348272667569519/tmp0dorw3fc.py')
PACKAGES=('anyio','cloudpickle','numpy','packaging','pandas','pyyaml','scikit-learn','snowflake-snowpark-python','typing-extensions','xgboost')
HANDLER='tmp0dorw3fc.infer'
</code>

<code>11) INSERT INTO "MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."_SYSTEM_REGISRTRY_DEPLOYMENTS" ( CREATION_TIME,DEPLOYMENT_NAME,MODEL_ID,OPTIONS,ROLE,SIGNATURE,STAGE_PATH,TARGET_METHOD,TARGET_PLATFORM ) SELECT CURRENT_TIMESTAMP(),'churn_model_predict','200e68963ff311ee89fa628aca6d24dc',OBJECT_CONSTRUCT('permanent_udf_stage_location','@"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"._SYSTEM_REGISRTRY_DEPLOYMENTS_STAGE/churn_model_predict/','relax_version',True),'"ACCOUNTADMIN"',OBJECT_CONSTRUCT('inputs',ARRAY_CONSTRUCT(OBJECT_CONSTRUCT('name','CUSTOMER_ID','type','INT64'),OBJECT_CONSTRUCT('name','CREDIT_SCORE','type','INT64'),OBJECT_CONSTRUCT('name','COUNTRY','type','STRING'),OBJECT_CONSTRUCT('name','GENDER','type','STRING'),OBJECT_CONSTRUCT('name','AGE','type','INT64'),OBJECT_CONSTRUCT('name','TENURE','type','INT64'),OBJECT_CONSTRUCT('name','BALANCE','type','DOUBLE'),OBJECT_CONSTRUCT('name','PRODUCTS_NUMBER','type','INT64'),OBJECT_CONSTRUCT('name','CREDIT_CARD','type','INT64'),OBJECT_CONSTRUCT('name','ACTIVE_MEMBER','type','INT64'),OBJECT_CONSTRUCT('name','ESTIMATED_SALARY','type','DOUBLE')),'outputs',ARRAY_CONSTRUCT(OBJECT_CONSTRUCT('name','PREDICTION','type','INT64'))),'@"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"._SYSTEM_REGISRTRY_DEPLOYMENTS_STAGE/churn_model_predict/','predict','warehouse'
</code>

<code>12) SELECT  *  FROM (SELECT * FROM "MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."_SYSTEM_REGISTRY_MODELS_VIEW") WHERE ("ID" = '200e68963ff311ee89fa628aca6d24dc')
</code>

<code>13) INSERT INTO "MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."_SYSTEM_REGISTRY_METADATA" ( ATTRIBUTE_NAME,EVENT_ID,EVENT_TIMESTAMP,MODEL_ID,OPERATION,ROLE,VALUE ) SELECT 'DEPLOYMENTS','c0d654603ffc11ee9f40628aca6d24dc',CURRENT_TIMESTAMP(),'200e68963ff311ee89fa628aca6d24dc','ADD','"ACCOUNTADMIN"',OBJECT_CONSTRUCT('DEPLOYMENTS',OBJECT_CONSTRUCT('name','churn_model_predict','permanent',True))
</code>

### Test

In [28]:
#load the sample data
table_name = 'BANK_CUSTOMER_CHURN_PREDICTION'
sf_df = session.table(table_name).drop('ROW')
train_df, test_df = sf_df.random_split([0.9,0.1])

In [34]:
result_dataframe = ppln_loaded.predict("churn_model_predict", test_df)

/opt/conda/lib/python3.8/site-packages/snowflake/ml/model/model_signature.py:1001: RuntimeWarning: Warn in feature CUSTOMER_ID: Nullable column CUSTOMER_ID provided, inference might fail if there is null value.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/snowflake/ml/model/model_signature.py:1001: RuntimeWarning: Warn in feature CREDIT_SCORE: Nullable column CREDIT_SCORE provided, inference might fail if there is null value.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/snowflake/ml/model/model_signature.py:1001: RuntimeWarning: Warn in feature COUNTRY: Nullable column COUNTRY provided, inference might fail if there is null value.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/snowflake/ml/model/model_signature.py:1001: RuntimeWarning: Warn in feature GENDER: Nullable column GENDER provided, inference might fail if there is null value.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/snowflake/ml/model/model_signature.py:1001: RuntimeWarning: Warn 

In [32]:
result_dataframe

In [33]:
result_dataframe.show()

----------------
|"PREDICTION"  |
----------------
|0             |
|0             |
|0             |
|0             |
|1             |
|1             |
|0             |
|0             |
|0             |
|1             |
----------------

